<a href="https://colab.research.google.com/github/himanshu-dutta/mb-object-detection-yolov5/blob/master/yolov5-training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Mon Feb 15 12:30:03 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!sudo apt-get install tree -qq > /dev/null
!pip install git+https://github.com/cocodataset/cocoapi.git
!git clone https://github.com/NVIDIA/apex && cd apex && pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" . --user && cd .. && rm -rf apex

debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-76edicbk
  Running command git clone -q https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-76edicbk
ERROR: Command errored out with exit status 1: python setup.py egg_info Check the logs for full command output.
Cloning into 'apex'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 7891 (delta 0), reused 0 (delta 0), pack-reused 7885
Receiving objects: 100% (7891/7891

In [3]:
!git clone https://github.com/ultralytics/yolov5.git
!pip install -r yolov5/requirements.txt -q

Cloning into 'yolov5'...
remote: Enumerating objects: 30, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 4809 (delta 13), reused 16 (delta 5), pack-reused 4779
Receiving objects: 100% (4809/4809), 7.80 MiB | 34.45 MiB/s, done.
Resolving deltas: 100% (3274/3274), done.
     |████████████████████████████████| 645kB 12.9MB/s 


# Setting up the dataset and the config files

### {assuming g-drive is connected to the nb}

In [4]:
!unzip -qq /content/drive/MyDrive/Projects/food-ingredients/top-10/data.zip -d /content/
!cp  /content/drive/MyDrive/Projects/food-ingredients/top-10/food_data.yaml /content/yolov5/data/
!cp /content/drive/MyDrive/Projects/food-ingredients/top-10/yolov5m.yaml /content/yolov5/models/

In [5]:
# directory structure for the data
!tree data -L 2

data
├── images
│   ├── train
│   └── val
└── labels
    ├── train
    └── val

6 directories, 0 files


In [6]:
# yolo format for the bounding box information
!cat "/content/data/labels/train/curry leaves_05ec7e800b.txt"

0 0.556225 0.492574 0.887550 0.727723


# Fine-tuning YOLOv5 model

### {yolov5l is being used as of now}

In [ ]:
%cd yolov5

/content/yolov5


In [ ]:
TRAIN_NAME = "food_ingredients_yolov5m"

!python train.py --img 640 --batch 16 --epochs 100 \
  --data ./data/food_data.yaml --cfg ./models/yolov5m.yaml --weights yolov5m.pt \
  --name {TRAIN_NAME} --cache

github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 v4.0-77-g3e560e2 torch 1.7.0+cu101 CUDA:0 (Tesla T4, 15109.75MB)

Namespace(adam=False, batch_size=16, bucket='', cache_images=True, cfg='./models/yolov5m.yaml', data='./data/food_data.yaml', device='', epochs=100, evolve=False, exist_ok=False, global_rank=-1, hyp='data/hyp.scratch.yaml', image_weights=False, img_size=[640, 640], linear_lr=False, local_rank=-1, log_artifacts=False, log_imgs=16, multi_scale=False, name='food_ingredients_yolov5m', noautoanchor=False, nosave=False, notest=False, project='runs/train', quad=False, rect=False, resume=False, save_dir='runs/train/food_ingredients_yolov5m', single_cls=False, sync_bn=False, total_batch_size=16, weights='yolov5m.pt', workers=8, world_size=1)
wandb: Install Weights & Biases for YOLOv5 logging with 'pip install wandb' (recommended)
Start Tensorboard with "tensorboard --logdir runs/train", view at http://localhost:6006/
2021-02-13 23:36:04.698690: I tensorflow/st

# Exporting to TorchScript & Onnx model

### Requires a few changes in the export file copying methods related to inference for further use.Refer to [the endpoint repository](https://github.com/himanshu-dutta/mbc-endpoint).

In [ ]:
!pip install onnx>=1.8.0 -q

In [ ]:
!python models/export.py --weights runs/train/food_ingredients_yolov5m/weights/best.pt --img 640 --batch 1

Namespace(batch_size=1, img_size=[640, 640], weights='runs/train/food_ingredients_yolov5m/weights/best.pt')
Fusing layers... 
Model Summary: 308 layers, 21074007 parameters, 0 gradients, 50.4 GFLOPS
<class 'tuple'>
2
torch.Size([1, 25200, 15]) <class 'torch.Tensor'>

Starting TorchScript export with torch 1.7.0+cu101...
./models/yolo.py:53: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if check0_ or check1_:
/usr/local/lib/python3.6/dist-packages/torch/jit/_trace.py:940: TracerWarning: Encountering a list at the output of the tracer might cause the trace to be incorrect, this is only valid if the container structure does not change based on the module's inputs. Consider using a constant container instead (e.g. for `list`, use a `tuple` instead. for `dict`, use a `Named

In [ ]:
# exporting the run to g-drive
!cp -R /content/yolov5/runs/train/food_ingredients_yolov5m /content/drive/MyDrive/Projects/food-ingredients/top-10/